## Data Question 4: Nashville COVID-19 Violations

In this project, you will explore COVID-19 violations reported to hubNashville, Metro Nashville government's comprehensive customer service system. 

In [1]:
#!pip install usaddress
import requests
import pandas as pd
pd.set_option('display.max_columns', 100)
pd.set_option('display.max_rows', 100)
import usaddress
from fuzzywuzzy import fuzz
from pandas import DataFrame
import numpy as np

C:\Users\duant\anaconda3\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')



### Part 1:
Gather the dataset of hubNashville requests from https://data.nashville.gov/Public-Services/hubNashville-311-Service-Requests/7qhx-rexh. For this project, you should look at requests with Request Type of "COVID-19" and Subrequest Type of "COVID-19 Violations". Explore this dataset, looking at when and where these violations occurred.


In [2]:
data=pd.read_csv('data/hubNashville__311__Service_Requests.csv')
data = data.fillna('nan')

C:\Users\duant\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
data.head(2)

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,System of Origin ID,Contact Type,Preferred Language,State Issue,Parent Request,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
0,113016,Closed,"Trash, Recycling & Litter",Cart Service,Trash - Request an Additional Cart,10/30/2018 01:47:00 PM,03/05/2019 09:35:00 PM,Phone,nan,nan,nan,False,nan,False,2009 Upland Dr,NASHVILLE,7,37216,36.1986,-86.7067,"(36.1986378, -86.7066586)"
1,53502,Closed,Property Violations,Residential Complaint,Residential Complaint,03/13/2018 09:34:50 PM,04/06/2018 01:13:32 PM,Phone,nan,nan,nan,False,nan,False,631 Murfreesboro Pike,NASHVILLE,17,37210,36.1401,-86.7384,"(36.14007, -86.73836)"


In [4]:
violation = data[(data['Request Type']=='COVID-19')&(data['Subrequest Type']=='COVID-19 Violations')]

In [5]:
violation.tail(2)

,Request #,Status,Request Type,Subrequest Type,Additional Subrequest Type,Date / Time Opened,Date / Time Closed,Request Origin,System of Origin ID,Contact Type,Preferred Language,State Issue,Parent Request,Closed When Created,Address,City,Council District,ZIP,Latitude,Longitude,Mapped Location
479105,492410,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,10/30/2020 09:47:56 PM,11/02/2020 03:50:18 PM,Phone,nan,nan,nan,False,nan,False,2408 Music Valley Dr,NASHVILLE,15,37214,36.2182,-86.698,"(36.21824, -86.69795)"
479249,491755,Closed,COVID-19,COVID-19 Violations,COVID-19 Violations,10/30/2020 03:50:56 AM,10/30/2020 01:57:59 PM,hubNashville Community,nan,nan,nan,False,nan,False,2500 Charlotte Ave,NASHVILLE,21,37209,36.1551,-86.8141,"(36.155145311603604, -86.8141005932541)"


In [6]:
violation.shape

(11632, 21)

In [7]:
violation['City'] =[i.title() for i in violation['City']]
violation['Add_City'] = violation['Address']+', ' +violation['City']

C:\Users\duant\anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
C:\Users\duant\anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [8]:
violation.to_csv('violation_clean.csv')

In [9]:
violation_add= violation.groupby(['Add_City']).count()['Request #'].sort_values()
type(violation_add)

pandas.core.series.Series

In [10]:
violation_nash = violation_add.to_frame().reset_index()
violation_nash.head()

,Add_City,Request #
0,"1 Averitt Express Dr, Nashville",1
1,"4130 Lebanon Pike, Hermitage",1
2,"414 Gallatin Pike S, Madison",1
3,"414 McNally Dr, Nashville",1
4,"414 Union St, Nashville",1


### Part 2:
The file davidson_cases.csv contains the number of COVID cases in Davidson county per day from March 8 through October 29. Use this dataset to compare the trend for the number of cases over time to the number of reported violations. Feel free to add any additional data that you think might be useful.


In [11]:
davidson_cases=pd.read_csv('data/davidson_cases.csv');

In [12]:
davidson_cases.head(5)

,date,county,state,fips,cases,deaths
0,2020-03-08,Davidson,Tennessee,47037,1,0
1,2020-03-09,Davidson,Tennessee,47037,1,0
2,2020-03-10,Davidson,Tennessee,47037,1,0
3,2020-03-11,Davidson,Tennessee,47037,2,0
4,2020-03-12,Davidson,Tennessee,47037,6,0



### Part 3:
The Metro Public Health Department tracks COVID-19 clusters. The files `clusters.csv` and `clusters_by_type.csv` contain the tables of clusters as reported by [WSMV](https://www.wsmv.com/news/metro-health-releases-latest-covid-19-clusters/article_ef554e08-1558-11eb-b290-873345e174d7.html) along with the coordinates of the clusters. Can you find any connection between the reported COVID violations and subsequent COVID clusters?


In [13]:
clusters=pd.read_csv('data/clusters.csv')

In [14]:
clusters.head()

,Cluster Name,Facility Type,Cluster Start Date,# Cases,Latitude,Longitude
0,Vanderbilt Parties,Social Gathering,3/11/2020,49,36.125891,-86.822863
1,Event at Clementine Hall,Social Gathering,3/14/2020,23,36.152444,-86.8467716
2,Religious Retreat,Social Gathering,3/25/2020,18,NaN,NaN
3,The Health Center at Richland Place,LTCF,4/3/2020,47,36.128750,-86.8195333
4,Trevecca Center for Rehab and Healing,LTCF,4/4/2020,102,36.144562,-86.7567485


In [15]:
clusters_by_type=pd.read_csv('data/clusters_by_type.csv')

In [16]:
clusters_by_type.head()

,Cluster Type,Number of Clusters
0,Bar,7
1,Church,2
2,College / University,5
3,Commercial-Warehouse,12
4,Congregate Living,13


### Part 4:
The dataset from data.nashville.gov includes geospatial information, which allows you to see where violations occurred geographically, but it does not provide information in regard to the specific businesses that were reported. In this part, you should explore the  businesses and types of businesses that have been reported. To attempt to answer this, you have been provided data from the [Google Places API](https://developers.google.com/places/web-service/overview). Each file is formatted as a json. The values are as follows:
* `mapped_location`: The mapped location from the hubNashville dataset
* `address`: The address from the hubNashville dataset
* `results`: The first five results from a Google Maps API nearbysearch, ranked by proximity to the Mapped Location. See https://developers.google.com/places/web-service/search#PlaceSearchResponses for more details on the fields in the results.

Take these results and use them to match as many violations as possible to a business. **Warning:** it will not be possible to match all violations to a business, and there will be ambiguities when trying to do this match, so do not spend all of your time on this task.
What do you find when you look into the types of businesses that have been reported for COVID violations?

https://maps.googleapis.com/maps/api/place/nearbysearch/json?location=36.0685457,-86.6838975&radius=10&type=point_of_interest&key=MYKEY

In [17]:
import glob
import json
from pandas.io.json import json_normalize

In [18]:
list_of_file=glob.glob('data/google_places_results/*.json')

In [19]:
df = pd.DataFrame() 

In [20]:
for i in list_of_file:
    df_new = pd.read_json(i)
    df = pd.concat([df,df_new])

In [21]:
df.tail(3)

,mapped_location,address,results
26,"(36.18837, -86.7676)","1001 Meridian St, Nashville","[{'business_status': 'OPERATIONAL', 'geometry'..."
27,"(36.1458305, -86.8167581)","221 31st Ave N, Nashville","[{'business_status': 'OPERATIONAL', 'geometry'..."
28,"(36.08778474206125, -86.72961779427743)","3806 Nolensville Pike, Nashville","[{'business_status': 'OPERATIONAL', 'geometry'..."


In [22]:
df.shape

(4829, 3)

In [23]:
df=df.reset_index(drop=True)

In [24]:
#df row 1, result 1/5.
df.iloc[0]['results'][0]

{'business_status': 'OPERATIONAL',
 'geometry': {'location': {'lat': 36.0685457, 'lng': -86.6838975},
  'viewport': {'northeast': {'lat': 36.06998608029151,
    'lng': -86.6823848697085},
   'southwest': {'lat': 36.06728811970851, 'lng': -86.6850828302915}}},
 'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/bar-71.png',
 'name': 'Los Paisanos',
 'opening_hours': {'open_now': True},
 'place_id': 'ChIJ7w-yGUFuZIgR7rrQpRfCGgw',
 'plus_code': {'compound_code': '3898+CC Nashville, TN, USA',
  'global_code': '868M3898+CC'},
 'rating': 4.2,
 'reference': 'ChIJ7w-yGUFuZIgR7rrQpRfCGgw',
 'scope': 'GOOGLE',
 'types': ['night_club', 'bar', 'point_of_interest', 'establishment'],
 'user_ratings_total': 6,
 'vicinity': '2045 Antioch Pike, Antioch'}

In [25]:
name1 = df.iloc[0]['results'][0]['name']
name1

'Los Paisanos'

In [26]:
for a in range (5):
    vars()['name'+str(a+1)]=[]
    vars()['add'+str(a+1)]=[]
    vars()['types'+str(a+1)]=[]

In [27]:
for i in range (4829):
    for a in range (5):
        name=df.iloc[i]['results'][a]['name']
        vars()['name'+str(a+1)].append(name)
        
        add=df.iloc[i]['results'][a]['vicinity']
        vars()['add'+str(a+1)].append(add)
        
        types=df.iloc[i]['results'][a]['types']
        vars()['types'+str(a+1)].append(types)


In [28]:
business_5 = pd.DataFrame(
    {'business_name1': name1,
     'business_address1': add1,
     'business_types1': types1,
     'business_name2': name2,
     'business_address2': add2,
     'business_types2': types2,
     'business_name3': name3,
     'business_address3': add3,
     'business_types3': types3,
     'business_name4': name4,
     'business_address4': add4,
     'business_types4': types4,
     'business_name5': name5,
     'business_address5': add5,
     'business_types5': types5,
    })


In [29]:
business_5.head(3)

,business_name1,business_address1,business_types1,business_name2,business_address2,business_types2,business_name3,business_address3,business_types3,business_name4,business_address4,business_types4,business_name5,business_address5,business_types5
0,Los Paisanos,"2045 Antioch Pike, Antioch","[night_club, bar, point_of_interest, establish...",Sun Hair Salon,"2049 Antioch Pike, Antioch","[beauty_salon, hair_care, point_of_interest, e...",Solo Style LLC,"2037 Antioch Pike, Antioch","[clothing_store, point_of_interest, store, est...",Hai Woon Dai,"2051 Antioch Pike, Antioch","[restaurant, food, point_of_interest, establis...","Dr. Babajide A. Bamigboye, MD","2031 Antioch Pike, Antioch","[doctor, health, point_of_interest, establishm..."
1,Best Buy Express Kiosk,"2800 Opryland Dr, Nashville","[point_of_interest, establishment]",Starbucks,Nashville,"[cafe, food, point_of_interest, store, establi...",Imaging USA 2018,"Gaylord Opryland Resort & Convention Center, 2...","[lodging, point_of_interest, establishment]",fiddler,"2800 Opryland Drive, Nashville","[lodging, point_of_interest, establishment]",Swarovski at Gaylord Opryland,"2800 Opryland Drive, Nashville","[jewelry_store, point_of_interest, store, esta..."
2,White Bison Coffee,"2607 12th Avenue South, Nashville","[convenience_store, cafe, restaurant, food, po...",Twice Daily,"2607 12th Avenue South, Nashville","[convenience_store, bakery, cafe, liquor_store...",Portland Brew,"2605 12th Avenue South, Nashville","[cafe, food, point_of_interest, establishment]",Alyn Vaughn,"2611 12th Avenue South, Nashville","[point_of_interest, store, establishment]",Blue Monkey Shaved Ice,"2611 12th Avenue South, Nashville","[food, point_of_interest, store, establishment]"


### Normalized Data

In [30]:
business_5 = df.join(business_5)
business_5.head()

,mapped_location,address,results,business_name1,business_address1,business_types1,business_name2,business_address2,business_types2,business_name3,business_address3,business_types3,business_name4,business_address4,business_types4,business_name5,business_address5,business_types5
0,"(36.0685457, -86.6838975)","2045 Antioch Pike, Antioch","[{'business_status': 'OPERATIONAL', 'geometry'...",Los Paisanos,"2045 Antioch Pike, Antioch","[night_club, bar, point_of_interest, establish...",Sun Hair Salon,"2049 Antioch Pike, Antioch","[beauty_salon, hair_care, point_of_interest, e...",Solo Style LLC,"2037 Antioch Pike, Antioch","[clothing_store, point_of_interest, store, est...",Hai Woon Dai,"2051 Antioch Pike, Antioch","[restaurant, food, point_of_interest, establis...","Dr. Babajide A. Bamigboye, MD","2031 Antioch Pike, Antioch","[doctor, health, point_of_interest, establishm..."
1,"(36.211269429469425, -86.6943024067459)","2800 Opryland Dr, Nashville","[{'business_status': 'OPERATIONAL', 'geometry'...",Best Buy Express Kiosk,"2800 Opryland Dr, Nashville","[point_of_interest, establishment]",Starbucks,Nashville,"[cafe, food, point_of_interest, store, establi...",Imaging USA 2018,"Gaylord Opryland Resort & Convention Center, 2...","[lodging, point_of_interest, establishment]",fiddler,"2800 Opryland Drive, Nashville","[lodging, point_of_interest, establishment]",Swarovski at Gaylord Opryland,"2800 Opryland Drive, Nashville","[jewelry_store, point_of_interest, store, esta..."
2,"(36.1235, -86.79013)","2607 12th Ave S, Nashville","[{'business_status': 'OPERATIONAL', 'geometry'...",White Bison Coffee,"2607 12th Avenue South, Nashville","[convenience_store, cafe, restaurant, food, po...",Twice Daily,"2607 12th Avenue South, Nashville","[convenience_store, bakery, cafe, liquor_store...",Portland Brew,"2605 12th Avenue South, Nashville","[cafe, food, point_of_interest, establishment]",Alyn Vaughn,"2611 12th Avenue South, Nashville","[point_of_interest, store, establishment]",Blue Monkey Shaved Ice,"2611 12th Avenue South, Nashville","[food, point_of_interest, store, establishment]"
3,"(36.1941628, -86.8313936)","3200 Clarksville Pike, Nashville","[{'business_status': 'OPERATIONAL', 'geometry'...",TIGERMARKET #203,"3200 Clarksville Highway, Nashville","[gas_station, point_of_interest, establishment]",Clarksville Pike & S Hamilton Rd SB,United States,"[transit_station, point_of_interest, establish...",Shoe Show,"Bordeaux S, 3209 Clarksville Pike C, Nashville","[shoe_store, point_of_interest, store, establi...",Excel Builders of Tennessee,"2107 Buena Vista Pike, Nashville","[general_contractor, real_estate_agency, point...",Campus Rx,"2109 Buena Vista Pike, Nashville","[health, point_of_interest, establishment]"
4,"(36.15843971160494, -86.7880380629013)","20 Grundy St, Nashville","[{'business_status': 'OPERATIONAL', 'geometry'...",Baldwin Showcase,"Grundy Street, Nashville","[point_of_interest, establishment]",16-Bit Bar+Arcade,"1102 Grundy Street ste b, Nashville","[bar, point_of_interest, establishment]",Christy Weaver,"1102 Grundy Street #A, Nashville","[point_of_interest, establishment]",Pins Mechanical Company,"1102 Grundy Street, Nashville","[bar, point_of_interest, establishment]",Due West Professional Building,"1101 Grundy Street, Nashville","[general_contractor, point_of_interest, establ..."


### Rules: 
- If normalized address number is not 100% match, then fail match;
- If match to more than 1 business, then fail match;
- If final fuzzy match score is below 70, then fail match;
- Else: find a match. 

In [31]:
business_5['address_norm']  = [usaddress.parse(i) for i in business_5.address] 
business_5['business_add1_norm']  = [usaddress.parse(i) for i in business_5.business_address1] 
business_5['business_add2_norm']  = [usaddress.parse(i) for i in business_5.business_address2] 
business_5['business_add3_norm']  = [usaddress.parse(i) for i in business_5.business_address3] 
business_5['business_add4_norm']  = [usaddress.parse(i) for i in business_5.business_address4] 
business_5['business_add5_norm']  = [usaddress.parse(i) for i in business_5.business_address5] 

In [32]:
business_5.head()

,mapped_location,address,results,business_name1,business_address1,business_types1,business_name2,business_address2,business_types2,business_name3,business_address3,business_types3,business_name4,business_address4,business_types4,business_name5,business_address5,business_types5,address_norm,business_add1_norm,business_add2_norm,business_add3_norm,business_add4_norm,business_add5_norm
0,"(36.0685457, -86.6838975)","2045 Antioch Pike, Antioch","[{'business_status': 'OPERATIONAL', 'geometry'...",Los Paisanos,"2045 Antioch Pike, Antioch","[night_club, bar, point_of_interest, establish...",Sun Hair Salon,"2049 Antioch Pike, Antioch","[beauty_salon, hair_care, point_of_interest, e...",Solo Style LLC,"2037 Antioch Pike, Antioch","[clothing_store, point_of_interest, store, est...",Hai Woon Dai,"2051 Antioch Pike, Antioch","[restaurant, food, point_of_interest, establis...","Dr. Babajide A. Bamigboye, MD","2031 Antioch Pike, Antioch","[doctor, health, point_of_interest, establishm...","[(2045, AddressNumber), (Antioch, StreetName),...","[(2045, AddressNumber), (Antioch, StreetName),...","[(2049, AddressNumber), (Antioch, StreetName),...","[(2037, AddressNumber), (Antioch, StreetName),...","[(2051, AddressNumber), (Antioch, StreetName),...","[(2031, AddressNumber), (Antioch, StreetName),..."
1,"(36.211269429469425, -86.6943024067459)","2800 Opryland Dr, Nashville","[{'business_status': 'OPERATIONAL', 'geometry'...",Best Buy Express Kiosk,"2800 Opryland Dr, Nashville","[point_of_interest, establishment]",Starbucks,Nashville,"[cafe, food, point_of_interest, store, establi...",Imaging USA 2018,"Gaylord Opryland Resort & Convention Center, 2...","[lodging, point_of_interest, establishment]",fiddler,"2800 Opryland Drive, Nashville","[lodging, point_of_interest, establishment]",Swarovski at Gaylord Opryland,"2800 Opryland Drive, Nashville","[jewelry_store, point_of_interest, store, esta...","[(2800, AddressNumber), (Opryland, StreetName)...","[(2800, AddressNumber), (Opryland, StreetName)...","[(Nashville, PlaceName)]","[(Gaylord, BuildingName), (Opryland, BuildingN...","[(2800, AddressNumber), (Opryland, StreetName)...","[(2800, AddressNumber), (Opryland, StreetName)..."
2,"(36.1235, -86.79013)","2607 12th Ave S, Nashville","[{'business_status': 'OPERATIONAL', 'geometry'...",White Bison Coffee,"2607 12th Avenue South, Nashville","[convenience_store, cafe, restaurant, food, po...",Twice Daily,"2607 12th Avenue South, Nashville","[convenience_store, bakery, cafe, liquor_store...",Portland Brew,"2605 12th Avenue South, Nashville","[cafe, food, point_of_interest, establishment]",Alyn Vaughn,"2611 12th Avenue South, Nashville","[point_of_interest, store, establishment]",Blue Monkey Shaved Ice,"2611 12th Avenue South, Nashville","[food, point_of_interest, store, establishment]","[(2607, AddressNumber), (12th, StreetName), (A...","[(2607, AddressNumber), (12th, StreetName), (A...","[(2607, AddressNumber), (12th, StreetName), (A...","[(2605, AddressNumber), (12th, StreetName), (A...","[(2611, AddressNumber), (12th, StreetName), (A...","[(2611, AddressNumber), (12th, StreetName), (A..."
3,"(36.1941628, -86.8313936)","3200 Clarksville Pike, Nashville","[{'business_status': 'OPERATIONAL', 'geometry'...",TIGERMARKET #203,"3200 Clarksville Highway, Nashville","[gas_station, point_of_interest, establishment]",Clarksville Pike & S Hamilton Rd SB,United States,"[transit_station, point_of_interest, establish...",Shoe Show,"Bordeaux S, 3209 Clarksville Pike C, Nashville","[shoe_store, point_of_interest, store, establi...",Excel Builders of Tennessee,"2107 Buena Vista Pike, Nashville","[general_contractor, real_estate_agency, point...",Campus Rx,"2109 Buena Vista Pike, Nashville","[health, point_of_interest, establishment]","[(3200, AddressNumber), (Clarksville, StreetNa...","[(3200, AddressNumber), (Clarksville, StreetNa...","[(United, CountryName), (States, CountryName)]","[(Bordeaux, SubaddressType), (S,, SubaddressId...","[(2107, AddressNumber), (Buena, Str

In [33]:
business_5['add1_match_num']  = business_5.apply(lambda x : 1 if x['business_add1_norm'][:1]==x['address_norm'][:1]  else 0, axis=1)
business_5['add2_match_num']  = business_5.apply(lambda x : 1 if x['business_add2_norm'][:1]==x['address_norm'][:1]  else 0, axis=1) 
business_5['add3_match_num']  = business_5.apply(lambda x : 1 if x['business_add3_norm'][:1]==x['address_norm'][:1]  else 0, axis=1)
business_5['add4_match_num']  = business_5.apply(lambda x : 1 if x['business_add4_norm'][:1]==x['address_norm'][:1]  else 0, axis=1) 
business_5['add5_match_num']  = business_5.apply(lambda x : 1 if x['business_add5_norm'][:1]==x['address_norm'][:1]  else 0, axis=1) 

In [34]:
business_5

,mapped_location,address,results,business_name1,business_address1,business_types1,business_name2,business_address2,business_types2,business_name3,business_address3,business_types3,business_name4,business_address4,business_types4,business_name5,business_address5,business_types5,address_norm,business_add1_norm,business_add2_norm,business_add3_norm,business_add4_norm,business_add5_norm,add1_match_num,add2_match_num,add3_match_num,add4_match_num,add5_match_num
0,"(36.0685457, -86.6838975)","2045 Antioch Pike, Antioch","[{'business_status': 'OPERATIONAL', 'geometry'...",Los Paisanos,"2045 Antioch Pike, Antioch","[night_club, bar, point_of_interest, establish...",Sun Hair Salon,"2049 Antioch Pike, Antioch","[beauty_salon, hair_care, point_of_interest, e...",Solo Style LLC,"2037 Antioch Pike, Antioch","[clothing_store, point_of_interest, store, est...",Hai Woon Dai,"2051 Antioch Pike, Antioch","[restaurant, food, point_of_interest, establis...","Dr. Babajide A. Bamigboye, MD","2031 Antioch Pike, Antioch","[doctor, health, point_of_interest, establishm...","[(2045, AddressNumber), (Antioch, StreetName),...","[(2045, AddressNumber), (Antioch, StreetName),...","[(2049, AddressNumber), (Antioch, StreetName),...","[(2037, AddressNumber), (Antioch, StreetName),...","[(2051, AddressNumber), (Antioch, StreetName),...","[(2031, AddressNumber), (Antioch, StreetName),...",1,0,0,0,0
1,"(36.211269429469425, -86.6943024067459)","2800 Opryland Dr, Nashville","[{'business_status': 'OPERATIONAL', 'geometry'...",Best Buy Express Kiosk,"2800 Opryland Dr, Nashville","[point_of_interest, establishment]",Starbucks,Nashville,"[cafe, food, point_of_interest, store, establi...",Imaging USA 2018,"Gaylord Opryland Resort & Convention Center, 2...","[lodging, point_of_interest, establishment]",fiddler,"2800 Opryland Drive, Nashville","[lodging, point_of_interest, establishment]",Swarovski at Gaylord Opryland,"2800 Opryland Drive, Nashville","[jewelry_store, point_of_interest, store, esta...","[(2800, AddressNumber), (Opryland, StreetName)...","[(2800, AddressNumber), (Opryland, StreetName)...","[(Nashville, PlaceName)]","[(Gaylord, BuildingName), (Opryland, BuildingN...","[(2800, AddressNumber), (Opryland, StreetName)...","[(2800, AddressNumber), (Opryland, StreetName)...",1,0,0,1,1
2,"(36.1235, -86.79013)","2607 12th Ave S, Nashville","[{'business_status': 'OPERATIONAL', 'geometry'...",White Bison Coffee,"2607 12th Avenue South, Nashville","[convenience_store, cafe, restaurant, food, po...",Twice Daily,"2607 12th Avenue South, Nashville","[convenience_store, bakery, cafe, liquor_store...",Portland Brew,"2605 12th Avenue South, Nashville","[cafe, food, point_of_interest, establishment]",Alyn Vaughn,"2611 12th Avenue South, Nashville","[point_of_interest, store, establishment]",Blue Monkey Shaved Ice,"2611 12th Avenue South, Nashville","[food, point_of_interest, store, establishment]","[(2607, AddressNumber), (12th, StreetName), (A...","[(2607, AddressNumber), (12th, StreetName), (A...","[(2607, AddressNumber), (12th, StreetName), (A...","[(2605, AddressNumber), (12th, StreetName), (A...","[(2611, AddressNumber), (12th, StreetName), (A...","[(2611, AddressNumber), (12th, StreetName), (A...",1,1,0,0,0
3,"(36.1941628, -86.8313936)","3200 Clarksville Pike, Nashville","[{'business_status': 'OPERATIONAL', 'geometry'...",TIGERMARKET #203,"3200 Clarksville Highway, Nashville","[gas_station, point_of_interest, establishment]",Clarksville Pike & S Hamilton Rd SB,United States,"[transit_station, point_of_interest, establish...",Shoe Show,"Bordeaux S, 3209 Clarksville Pike C, Nashville","[shoe_store, point_of_interest, store, establi...",Excel Builders of Tennessee,"2107 Buena Vista Pike, Nashville","[general_contractor, real_estate_agency, point...",Campus Rx,"2109 Buena Vista Pike, Nashville","[health, point_of_interest, establishment]","[(3200, AddressNumber), (Clarksville, StreetNa...","[(3200, AddressNumber), (Clarksville, StreetNa...","[(United, CountryName), (States

In [35]:
business_5.to_csv('cleaned_data.csv',index=False)

In [36]:
business_5['final'] = business_5['add1_match_num']+business_5['add2_match_num']+business_5['add3_match_num']+business_5['add4_match_num']+business_5['add5_match_num']

In [37]:
business_5['final'].value_counts()

1    1781
0    1412
2     637
5     411
3     329
4     259
Name: final, dtype: int64

In [53]:
business_5

,mapped_location,address,results,business_name1,business_address1,business_types1,business_name2,business_address2,business_types2,business_name3,business_address3,business_types3,business_name4,business_address4,business_types4,business_name5,business_address5,business_types5,address_norm,business_add1_norm,business_add2_norm,business_add3_norm,business_add4_norm,business_add5_norm,add1_match_num,add2_match_num,add3_match_num,add4_match_num,add5_match_num,final
0,"(36.0685457, -86.6838975)","2045 Antioch Pike, Antioch","[{'business_status': 'OPERATIONAL', 'geometry'...",Los Paisanos,"2045 Antioch Pike, Antioch","[night_club, bar, point_of_interest, establish...",Sun Hair Salon,"2049 Antioch Pike, Antioch","[beauty_salon, hair_care, point_of_interest, e...",Solo Style LLC,"2037 Antioch Pike, Antioch","[clothing_store, point_of_interest, store, est...",Hai Woon Dai,"2051 Antioch Pike, Antioch","[restaurant, food, point_of_interest, establis...","Dr. Babajide A. Bamigboye, MD","2031 Antioch Pike, Antioch","[doctor, health, point_of_interest, establishm...","[(2045, AddressNumber), (Antioch, StreetName),...","[(2045, AddressNumber), (Antioch, StreetName),...","[(2049, AddressNumber), (Antioch, StreetName),...","[(2037, AddressNumber), (Antioch, StreetName),...","[(2051, AddressNumber), (Antioch, StreetName),...","[(2031, AddressNumber), (Antioch, StreetName),...",1,0,0,0,0,1
1,"(36.211269429469425, -86.6943024067459)","2800 Opryland Dr, Nashville","[{'business_status': 'OPERATIONAL', 'geometry'...",Best Buy Express Kiosk,"2800 Opryland Dr, Nashville","[point_of_interest, establishment]",Starbucks,Nashville,"[cafe, food, point_of_interest, store, establi...",Imaging USA 2018,"Gaylord Opryland Resort & Convention Center, 2...","[lodging, point_of_interest, establishment]",fiddler,"2800 Opryland Drive, Nashville","[lodging, point_of_interest, establishment]",Swarovski at Gaylord Opryland,"2800 Opryland Drive, Nashville","[jewelry_store, point_of_interest, store, esta...","[(2800, AddressNumber), (Opryland, StreetName)...","[(2800, AddressNumber), (Opryland, StreetName)...","[(Nashville, PlaceName)]","[(Gaylord, BuildingName), (Opryland, BuildingN...","[(2800, AddressNumber), (Opryland, StreetName)...","[(2800, AddressNumber), (Opryland, StreetName)...",1,0,0,1,1,3
2,"(36.1235, -86.79013)","2607 12th Ave S, Nashville","[{'business_status': 'OPERATIONAL', 'geometry'...",White Bison Coffee,"2607 12th Avenue South, Nashville","[convenience_store, cafe, restaurant, food, po...",Twice Daily,"2607 12th Avenue South, Nashville","[convenience_store, bakery, cafe, liquor_store...",Portland Brew,"2605 12th Avenue South, Nashville","[cafe, food, point_of_interest, establishment]",Alyn Vaughn,"2611 12th Avenue South, Nashville","[point_of_interest, store, establishment]",Blue Monkey Shaved Ice,"2611 12th Avenue South, Nashville","[food, point_of_interest, store, establishment]","[(2607, AddressNumber), (12th, StreetName), (A...","[(2607, AddressNumber), (12th, StreetName), (A...","[(2607, AddressNumber), (12th, StreetName), (A...","[(2605, AddressNumber), (12th, StreetName), (A...","[(2611, AddressNumber), (12th, StreetName), (A...","[(2611, AddressNumber), (12th, StreetName), (A...",1,1,0,0,0,2
3,"(36.1941628, -86.8313936)","3200 Clarksville Pike, Nashville","[{'business_status': 'OPERATIONAL', 'geometry'...",TIGERMARKET #203,"3200 Clarksville Highway, Nashville","[gas_station, point_of_interest, establishment]",Clarksville Pike & S Hamilton Rd SB,United States,"[transit_station, point_of_interest, establish...",Shoe Show,"Bordeaux S, 3209 Clarksville Pike C, Nashville","[shoe_store, point_of_interest, store, establi...",Excel Builders of Tennessee,"2107 Buena Vista Pike, Nashville","[general_contractor, real_estate_agency, point...",Campus Rx,"2109 Buena Vista Pike, Nashville","[health, point_of_interest, establishment]","[(3200, AddressNumber), (Clarksville, StreetNa...","[(3200, AddressNumber), (Clarksville, StreetNa...","[(United, CountryNa

In [38]:
business_final = business_5[business_5['final'] == 1].reset_index(drop=True)

In [39]:
business_final.loc[business_final.add1_match_num == 1, 'final_name'] = business_final.business_name1 
business_final.loc[business_final.add2_match_num == 1, 'final_name'] = business_final.business_name2
business_final.loc[business_final.add3_match_num == 1, 'final_name'] = business_final.business_name3
business_final.loc[business_final.add4_match_num == 1, 'final_name'] = business_final.business_name4
business_final.loc[business_final.add5_match_num == 1, 'final_name'] = business_final.business_name5

business_final.loc[business_final.add1_match_num == 1, 'final_address'] = business_final.business_address1 
business_final.loc[business_final.add2_match_num == 1, 'final_address'] = business_final.business_address2
business_final.loc[business_final.add3_match_num == 1, 'final_address'] = business_final.business_address3
business_final.loc[business_final.add4_match_num == 1, 'final_address'] = business_final.business_address4
business_final.loc[business_final.add5_match_num == 1, 'final_address'] = business_final.business_address5

business_final.loc[business_final.add1_match_num == 1, 'final_address_norm'] = business_final.business_add1_norm 
business_final.loc[business_final.add2_match_num == 1, 'final_address_norm'] = business_final.business_add2_norm
business_final.loc[business_final.add3_match_num == 1, 'final_address_norm'] = business_final.business_add3_norm
business_final.loc[business_final.add4_match_num == 1, 'final_address_norm'] = business_final.business_add4_norm
business_final.loc[business_final.add5_match_num == 1, 'final_address_norm'] = business_final.business_add5_norm

business_final.loc[business_final.add1_match_num == 1, 'final_types'] = business_final.business_types1 
business_final.loc[business_final.add2_match_num == 1, 'final_types'] = business_final.business_types2
business_final.loc[business_final.add3_match_num == 1, 'final_types'] = business_final.business_types3
business_final.loc[business_final.add4_match_num == 1, 'final_types'] = business_final.business_types4
business_final.loc[business_final.add5_match_num == 1, 'final_types'] = business_final.business_types5

In [40]:
business_final= business_final[['address','address_norm','final_name','final_address','final_address_norm','final_types']]

In [41]:
business_final['ratio']=business_final.apply(lambda x: fuzz.token_sort_ratio(x['address_norm'], x['final_address_norm']), axis=1)
business_final.head(3)

,address,address_norm,final_name,final_address,final_address_norm,final_types,ratio
0,"2045 Antioch Pike, Antioch","[(2045, AddressNumber), (Antioch, StreetName),...",Los Paisanos,"2045 Antioch Pike, Antioch","[(2045, AddressNumber), (Antioch, StreetName),...","[night_club, bar, point_of_interest, establish...",100
1,"3200 Clarksville Pike, Nashville","[(3200, AddressNumber), (Clarksville, StreetNa...",TIGERMARKET #203,"3200 Clarksville Highway, Nashville","[(3200, AddressNumber), (Clarksville, StreetNa...","[gas_station, point_of_interest, establishment]",92
2,"130 LifeWay Plaza, Nashville","[(130, AddressNumber), (LifeWay, StreetName), ...",Frankie Pierce Park,"130 LifeWay Plaza, Nashville","[(130, AddressNumber), (LifeWay, StreetName), ...","[park, point_of_interest, establishment]",100


In [42]:
business_final[business_final['ratio']>=70].sort_values('ratio')

,address,address_norm,final_name,final_address,final_address_norm,final_types,ratio
231,"235 TN-174, Goodlettsville","[(235, AddressNumber), (TN-174,, StreetName), ...",Cracker Barrel Old Country Store,"235 Long Hollow Pike, Goodlettsville","[(235, AddressNumber), (Long, StreetName), (Ho...","[restaurant, food, point_of_interest, store, e...",70
1559,"8129 Sawyer Brown Rd, Nashville","[(8129, AddressNumber), (Sawyer, StreetName), ...",DeSano Pizzeria Napoletana,"8129 Sawyer Brown Road Suite #401, Bellevue","[(8129, AddressNumber), (Sawyer, StreetName), ...","[restaurant, food, point_of_interest, establis...",71
98,"1299 Murfreesboro Rd, Nan","[(1299, AddressNumber), (Murfreesboro, StreetN...",McDonald's,"1299 Tennessee 96 E, Franklin","[(1299, AddressNumber), (Tennessee, StreetName...","[cafe, restaurant, food, point_of_interest, st...",71
609,"5011 TN-60, Nan","[(5011, AddressNumber), (TN-60,, StreetName), ...",Oasis Revival Center,"5011 Tennessee 60, Birchwood","[(5011, AddressNumber), (Tennessee, StreetName...","[church, place_of_worship, point_of_interest, ...",71
120,"8121 TN-100, Nashville","[(8121, AddressNumber), (TN-100,, StreetName),...",Starbucks,"8121 Tennessee 100, Bellevue","[(8121, AddressNumber), (Tennessee, StreetName...","[cafe, restaurant, food, point_of_interest, st...",72
...,...,...,...,...,...,...,...
998,"5406 Harding Pike, Nashville","[(5406, AddressNumber), (Harding, StreetName),...",Coco's West Italian Restaurant,"5406 Harding Pike, Nashville","[(5406, AddressNumber), (Harding, StreetName),...","[restaurant, food, point_of_interest, establis...",100
999,"3758 Nolensville Pike, Nashville","[(3758, AddressNumber), (Nolensville, StreetNa...",ALDI,"3758 Nolensville Pike, Nashville","[(3758, AddressNumber), (Nolensville, StreetNa...","[grocery_or_supermarket, supermarket, food, po...",100
1002,"2270 Lebanon Pike, Nashville","[(2270, AddressNumber), (Lebanon, StreetName),...",ChiroExpress,"2270 Lebanon Pike, Nashville","[(2270, AddressNumber), (Lebanon, StreetName),...","[doctor, health, point_of_interest, establishm...",100
1011,"4831 Nolensville Pike, Nashville","[(4831, AddressNumber), (Nolensville, StreetNa...",Dairy Queen Grill & Chill,"4831 Nolensville Pike, Nashville","[(4831, AddressNumber), (Nolensville, StreetNa...","[restaurant, food, point_of_interest, store, e...",100


In [43]:
business_final[business_final['ratio']<70].sort_values('ratio')

,address,address_norm,final_name,final_address,final_address_norm,final_types,ratio
1318,"306 TN-6, Nashville","[(306, AddressNumber), (TN-6,, StreetName), (N...",Fifth Third Bank & ATM,"306 North Gallatin Pike North, Madison","[(306, AddressNumber), (North, StreetNamePreDi...","[bank, atm, finance, point_of_interest, establ...",47
1497,"Rusty Nail, Hermitage","[(Rusty, Recipient), (Nail,, Recipient), (Herm...",The Rusty Nail Bar,"Rusty Nail, 4054 Andrew Jackson Way, Hermitage","[(Rusty, Recipient), (Nail,, Recipient), (4054...","[bar, night_club, restaurant, food, point_of_i...",49
1192,"135 US-31E, Madison","[(135, AddressNumber), (US-31E,, StreetName), ...",ALDI,"135 S Gallatin Pike North, Madison","[(135, AddressNumber), (S, StreetNamePreDirect...","[grocery_or_supermarket, supermarket, food, po...",50
902,"2787 Murfreesboro Pike, Antioch","[(2787, AddressNumber), (Murfreesboro, StreetN...",Sonic Drive-In,"2787 Murfreesboro Pike (Murfreesboro Pike &, M...","[(2787, AddressNumber), (Murfreesboro, StreetN...","[restaurant, food, point_of_interest, store, e...",54
176,"400 5th Ave N, Nashville","[(400, AddressNumber), (5th, StreetName), (Ave...",Dunkin',400 Charlotte Ave Music City Central Transit H...,"[(400, AddressNumber), (Charlotte, StreetName)...","[bakery, meal_takeaway, cafe, restaurant, food...",55
34,"1211 Medical Center Dr, Nashville","[(1211, AddressNumber), (Medical, StreetName),...",Eagle Susan S MD,"1211 22nd Ave S # 5220MCN, Nashville","[(1211, AddressNumber), (22nd, StreetName), (A...","[doctor, health, point_of_interest, establishm...",58
666,"7613 TN-1, Nashville","[(7613, AddressNumber), (TN-1,, StreetName), (...",Dairy Queen Grill & Chill,"7613 Highway 70 South, Bellevue","[(7613, AddressNumber), (Highway, StreetNamePr...","[restaurant, food, point_of_interest, store, e...",58
109,"1771 US-31E, Madison","[(1771, AddressNumber), (US-31E,, StreetName),...",Red Wing Shoes,"1771 Gallatin Pike North, Madison","[(1771, AddressNumber), (Gallatin, StreetName)...","[shoe_store, point_of_interest, store, establi...",59
1456,"4306 TN-66, Nan","[(4306, AddressNumber), (TN-66,, StreetName), ...",McDonald's,"4306 Tennessee 66 S, Rogersville","[(4306, AddressNumber), (Tennessee, StreetName...","[cafe, restaurant, food, point_of_interest, st...",59
205,"Anderson Rd & Smith Springs Rd, Nashville","[(Anderson, StreetName), (Rd, StreetNamePostTy...",S.M.T. Boutique & Things,"Anderson Road, Nashville","[(Anderson, StreetName), (Road,, StreetNamePos...","[point_of_interest, clothing_store, store, est...",60


### Most Violated Business

In [44]:
address_by_request= business_final[business_final['ratio']>=70].sort_values('ratio')

In [45]:
final= pd.merge(address_by_request, violation_nash, how='left', left_on='address', right_on='Add_City',sort=True).sort_values(by='Request #',ascending=False)
final.head(10)

,address,address_norm,final_name,final_address,final_address_norm,final_types,ratio,Add_City,Request #
504,"221 Broadway, Nashville","[(221, AddressNumber), (Broadway,, StreetName)...",Kid Rock's Big Ass Honky Tonk Rock N' Roll Ste...,"221 Broadway, Nashville","[(221, AddressNumber), (Broadway,, StreetName)...","[restaurant, food, point_of_interest, establis...",100,"221 Broadway, Nashville",46
584,"2408 Music Valley Dr, Nashville","[(2408, AddressNumber), (Music, StreetName), (...",Scoreboard Bar & Grill,"2408 Music Valley Drive, Nashville","[(2408, AddressNumber), (Music, StreetName), (...","[bar, point_of_interest, establishment]",98,"2408 Music Valley Dr, Nashville",43
1548,"717 3rd Ave N, Nashville","[(717, AddressNumber), (3rd, StreetName), (Ave...",Jonathan's Grille,"717 3rd Avenue North, Nashville","[(717, AddressNumber), (3rd, StreetName), (Ave...","[bar, restaurant, food, point_of_interest, est...",96,"717 3rd Ave N, Nashville",40
123,"1200 Donelson Ave, Old Hickory","[(1200, AddressNumber), (Donelson, StreetName)...",Chopper's Grill,"1200 Donelson Avenue, Old Hickory","[(1200, AddressNumber), (Donelson, StreetName)...","[bar, restaurant, food, point_of_interest, est...",98,"1200 Donelson Ave, Old Hickory",32
419,"2037 Antioch Pike, Antioch","[(2037, AddressNumber), (Antioch, StreetName),...",Solo Style LLC,"2037 Antioch Pike, Antioch","[(2037, AddressNumber), (Antioch, StreetName),...","[clothing_store, point_of_interest, store, est...",100,"2037 Antioch Pike, Antioch",29
1086,"422 Broadway, Nashville","[(422, AddressNumber), (Broadway,, StreetName)...",Tootsies Orchid Lounge,"422 Broadway, Nashville","[(422, AddressNumber), (Broadway,, StreetName)...","[tourist_attraction, bar, restaurant, food, po...",100,"422 Broadway, Nashville",27
380,"1913 Division St, Nashville","[(1913, AddressNumber), (Division, StreetName)...",Winners Bar & Grill,"1913 Division Street, Nashville","[(1913, AddressNumber), (Division, StreetName)...","[bar, restaurant, food, point_of_interest, est...",98,"1913 Division St, Nashville",26
1047,"411 Gallatin Pike S, Madison","[(411, AddressNumber), (Gallatin, StreetName),...",Super hair world,"411 Gallatin Pike South, Madison","[(411, AddressNumber), (Gallatin, StreetName),...","[hair_care, point_of_interest, store, establis...",98,"411 Gallatin Pike S, Madison",22
155,"1225 Northgate Business Pkwy, Madison","[(1225, AddressNumber), (Northgate, StreetName...",Metal Moulding Corporation,"1225 Northgate Business Parkway, Madison","[(1225, AddressNumber), (Northgate, StreetName...","[point_of_interest, establishment]",99,"1225 Northgate Business Pkwy, Madison",21
855,"3308 Gallatin Pike, Nashville","[(3308, AddressNumber), (Gallatin, StreetName)...",Sinkers Beverages,"3308 Gallatin Pike, Nashville","[(3308, AddressNumber), (Gallatin, StreetName)...","[liquor_store, point_of_interest, store, estab...",100,"3308 Gallatin Pike, Nashville",21


### Most Violated Type

In [46]:
final['type&request'] = final['Request #']*final['final_types']

In [47]:
final['type&request'].iloc[0][:20]

['restaurant',
 'food',
 'point_of_interest',
 'establishment',
 'restaurant',
 'food',
 'point_of_interest',
 'establishment',
 'restaurant',
 'food',
 'point_of_interest',
 'establishment',
 'restaurant',
 'food',
 'point_of_interest',
 'establishment',
 'restaurant',
 'food',
 'point_of_interest',
 'establishment']

In [48]:
types=[]

In [49]:
for i in final['type&request']:
    for j in i: 
        types.append(j)

In [50]:
from collections import Counter
import collections

In [51]:
most_common_type=collections.Counter(types).most_common()

In [52]:
violation_type = pd.DataFrame.from_records(most_common_type, columns =['violation', 'count']) 
violation_type[2:12]

,violation,count
2,food,1798
3,restaurant,1455
4,store,1159
5,bar,594
6,health,246
7,home_goods_store,221
8,meal_takeaway,212
9,grocery_or_supermarket,177
10,car_repair,138
11,cafe,111


In [54]:
import glob
import json
import pandas as pd
from pandas.io.json import json_normalize


In [55]:
from pandas.io.json import json_normalize
google_results = pd.DataFrame() #create empty dataframe
for filename in glob.glob('data/google_places_results/*.json'): # loop over .json files
    with open(filename) as f:
        google_files = json.load(f) # open the json file
        google_jsons = pd.json_normalize(google_files, sep='_', #normalize the json file for nested results
                            record_path='results')
        google_results = google_results.append(google_jsons) #append each json file results to df


In [56]:
google_results.head(3)

,business_status,icon,name,place_id,rating,reference,scope,types,user_ratings_total,vicinity,geometry_location_lat,geometry_location_lng,geometry_viewport_northeast_lat,geometry_viewport_northeast_lng,geometry_viewport_southwest_lat,geometry_viewport_southwest_lng,opening_hours_open_now,plus_code_compound_code,plus_code_global_code,photos,price_level,permanently_closed
0,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,Los Paisanos,ChIJ7w-yGUFuZIgR7rrQpRfCGgw,4.2,ChIJ7w-yGUFuZIgR7rrQpRfCGgw,GOOGLE,"[night_club, bar, point_of_interest, establish...",6.0,"2045 Antioch Pike, Antioch",36.068546,-86.683898,36.069986,-86.682385,36.067288,-86.685083,True,"3898+CC Nashville, TN, USA",868M3898+CC,NaN,NaN,NaN
1,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,Sun Hair Salon,ChIJlfxUC0FuZIgRa8G6Arf-BW8,4.6,ChIJlfxUC0FuZIgRa8G6Arf-BW8,GOOGLE,"[beauty_salon, hair_care, point_of_interest, e...",50.0,"2049 Antioch Pike, Antioch",36.068475,-86.683967,36.069942,-86.682412,36.067244,-86.685110,True,"3898+9C Nashville, TN, USA",868M3898+9C,"[{'height': 3024, 'html_attributions': ['<a hr...",NaN,NaN
2,OPERATIONAL,https://maps.gstatic.com/mapfiles/place_api/ic...,Solo Style LLC,ChIJl59cc0FuZIgRkvAljwlcGg0,4.0,ChIJl59cc0FuZIgRkvAljwlcGg0,GOOGLE,"[clothing_store, point_of_interest, store, est...",4.0,"2037 Antioch Pike, Antioch",36.068658,-86.683813,36.070064,-86.682361,36.067366,-86.685059,NaN,"3898+FF Nashville, TN, USA",868M3898+FF,NaN,NaN,NaN


In [57]:
pd.__version__


'1.0.1'